In [1]:
# LLM + LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.memory import ConversationBufferMemory
from prompt_config import prefix, suffix, few_shots
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI 
import os
from dotenv import load_dotenv



# Data handling
import pandas as pd

# Plotting
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px
from langchain.document_loaders import CSVLoader

# Utilities
import os


In [2]:

load_dotenv()
key= os.getenv("OPENAI_API_KEY")

# Initialize GPT-5 Nano
llm = ChatOpenAI(openai_api_key=key, model_name="gpt-5-mini", temperature=1 )





/var/folders/lk/23npv5w93yvfg4q5_4b_sq7h0000gn/T/ipykernel_15830/2938347930.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=key, model_name="gpt-5-mini", temperature=1 )


In [30]:
def decide_salary(df, gender_col: str, age_col: str) -> pd.DataFrame:
    def calc_salary(row):
        gender = str(row[gender_col]).lower()
        age = str(row[age_col]).lower()
        if gender == 'male' and age == 'young':
            return 75000
        elif gender == 'female' and age == 'young':
            return 100000
        else:
            return 50000

    df['Salary'] = df.apply(calc_salary, axis=1)
    return df


In [31]:
def clean_code(code: str) -> str:
    """
    Remove Markdown backticks and language hints from LLM-generated code.
    """
    code = code.strip()
    # Remove ```python or ``` at the start
    if code.startswith("```"):
        code = "\n".join(code.splitlines()[1:])
    # Remove ``` at the end
    if code.endswith("```"):
        code = "\n".join(code.splitlines()[:-1])
    return code

In [32]:
import pandas as pd

# Load your CSV
df = pd.read_csv("/Users/piratheebanmayuran/Documents/GitHub/pira_projects/llm_projects/data.csv")


In [36]:
def validate_examples(examples):
    for i, ex in enumerate(examples):
        for key in ["input", "output"]:
            if key not in ex:
                raise ValueError(f"Example {i} is missing the key '{key}'")
            if not isinstance(ex[key], str):
                # Convert lists or other types to string automatically
                print(f"Warning: Example {i} key '{key}' is not a string. Converting to string.")
                ex[key] = str(ex[key])
    return examples

In [38]:
validate_examples(few_shots)

[{'input': 'Show summary of Salary column',
  'output': "print(df['Salary'].describe())"},
 {'input': 'calculate the salary based on the gender and age of the employee',
  'output': "df = decide_salary(df, 'Gender', 'AgeCategory')"},
 {'input': 'show summary of the data', 'output': 'print(df.describe())'},
 {'input': 'Plot Gender distribution',
  'output': "fig = px.histogram(df, x='Gender')\nfig"},
 {'input': 'Create a bar chart of Age distribution',
  'output': "fig = px.bar(df['AgeCategory'].value_counts().reset_index(), x='index', y='AgeCategory')\nfig"},
 {'input': 'Show scatter plot of Age vs Salary',
  'output': "fig = px.scatter(df, x='AgeCategory', y='Salary', color='Gender')\nfig"}]

In [37]:
# Prompt forces model to return Python code using Pandas + Plotly
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    # Create texts for vectorization by combining input and output
to_vectorize = [f"{example['input']} {example['output']}" for example in few_shots]
cleaned_few_shots = [{k: str(v) for k, v in ex.items()} for ex in few_shots]
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=cleaned_few_shots)
example_selector = SemanticSimilarityExampleSelector(
        vectorstore=vectorstore,
        k=2,
    )
example_prompt = PromptTemplate(
        input_variables=["input", "output"],
        template="Human: {input}\nAI:\n{output}"
    )

prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["history", "input"]
)

memory = ConversationBufferMemory(memory_key="history")

conversation = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=True
)
print(cleaned_few_shots)

[{'input': 'Show summary of Salary column', 'output': "print(df['Salary'].describe())"}, {'input': 'calculate the salary based on the gender and age of the employee', 'output': "df = decide_salary(df, 'Gender', 'AgeCategory')"}, {'input': 'show summary of the data', 'output': 'print(df.describe())'}, {'input': 'Plot Gender distribution', 'output': "fig = px.histogram(df, x='Gender')\nfig"}, {'input': 'Create a bar chart of Age distribution', 'output': "fig = px.bar(df['AgeCategory'].value_counts().reset_index(), x='index', y='AgeCategory')\nfig"}, {'input': 'Show scatter plot of Age vs Salary', 'output': "fig = px.scatter(df, x='AgeCategory', y='Salary', color='Gender')\nfig"}]


In [34]:
# Example: user wants a category column based on gender
user_input = "Create a category column 'GenderCode' based on Gender: male=01, female=02"
response = conversation.invoke({"input": user_input})


# Example usage
user_code = response["response"]
user_code = clean_code(user_code)
exec(user_code)


# df is now updated in memory
print(df.head())




> Entering new ConversationChain chain...
Prompt after formatting:
 
    You are a Python data assistant.
    Rules:
    - if df is not present, do NOT attempt to load CSV; ask for path from user input to load CSV
    - Use pandas for data manipulations
    - Use plotly.express (px) for plotting
    - Always update 'df' in place
    - NEVER save CSV unless instructed
    - Only return clean runnable Python code
    - DO NOT include any Markdown, backticks, or explanations
    - Respond ONLY with the code
    - when user ask to calculate the salary based on the gender and age category, use the predefined function decide_salary(df, gender, age_category), look for columns name that has similar name like gender, age_group, age_category etc.
    - when user ask to show summary of the data, use df.describe() and print the result
    - If 'df' is not present, do NOT attempt to load CSV; wait for user input
    - For plotting: ALWAYS create Plotly figures with variable name 'fig' (e.g., fig 

DTypePromotionError: The DType <class 'numpy.dtypes.StrDType'> could not be promoted by <class 'numpy.dtypes._PyFloatDType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.StrDType'>, <class 'numpy.dtypes._PyFloatDType'>)

In [ ]:
user_input = "Create a category column 'IncomeCategory' based on Salary: <50000=low, >=500000=high"
response = conversation.invoke({"input": user_input})


# Example usage
user_code = response["response"]
user_code = clean_code(user_code)
exec(user_code)

In [ ]:
user_input = "Create a category column 'AgeCategory' based on Age: <30=young, else= old"
response = conversation.invoke({"input": user_input})


# Example usage
user_code = response["response"]
user_code = clean_code(user_code)
exec(user_code)

print(df.head())

In [ ]:
print(user_code)

In [ ]:
user_input = "calculate the salary based on the Gender and AgeCategory of the employee"
response = conversation.invoke({"input": user_input})
user_code = response["response"]
user_code = clean_code(user_code)
exec(user_code)

print(user_code)

print(df.head())

In [ ]:
user_input = "Plot Gender as a pie chart using plotly"
response = conversation.invoke({"input": user_input})
user_code = response["response"]
user_code = clean_code(user_code)
exec(user_code)

print(df.head())

In [ ]:
user_input = "change column name from AgeCategory to AgeGroup"
response = conversation.invoke({"input": user_input})

user_code = response["response"]
user_code = clean_code(user_code)
exec(user_code)

In [ ]:
print(df.head())

In [ ]:
# user_input = "save df as a csv file in the current directory"
# response = conversation.invoke({"input": user_input})

# user_code = response["response"]
# user_code = clean_code(user_code)
# exec(user_code)


In [ ]:
df